# Part 2: Train, Check Bias, Tune, Record Lineage, and Register a Model

<a id='aud-overview'> </a>

## [Overview](./0-AutoClaimFraudDetection.ipynb)
* [Notebook 0 : Overview, Architecture and Data Exploration](./0-AutoClaimFraudDetection.ipynb)
* [Notebook 1: Data Prep, Process, Store Features](./1-data-prep-e2e.ipynb)
* **[Notebook 2: Train, Check Bias, Tune, Record Lineage, and Register a Model](./2-lineage-train-assess-bias-tune-registry-e2e.ipynb)**
  * **[Architecture](#train)**
  * **[Train a model using XGBoost](#aud-train-model)**
  * **[Model lineage with artifacts and associations](#model-lineage)**
  * **[Evaluate the model for bias with Clarify](#check-bias)**
  * **[Deposit Model and Lineage in SageMaker Model Registry](#model-registry)**
* [Notebook 3: Mitigate Bias, Train New Model, Store in Registry](./3-mitigate-bias-train-model2-registry-e2e.ipynb)
* [Notebook 4: Deploy Model, Run Predictions](./4-deploy-run-inference-e2e.ipynb)
* [Notebook 5 : Create and Run an End-to-End Pipeline to Deploy the Model](./5-pipeline-e2e.ipynb)

이 섹션에서는 SageMaker Clarify를 사용하여 사전 훈련 및 사후 훈련 편향을 평가하고, SageMaker에서 XGBoost를 사용하여 모델을 훈련시킨 다음, 마지막으로 함께 생성된 아티팩트의 계보(lineage)와 함께 모델 레지스트리에 배치하는 방법을 보여줍니다. 

두 번째 모델에서는 SMOTE를 사용하여 데이터셋의 성별 불균형을 수정하고 XGBoost를 사용하여 다른 모델을 훈련합ㄷ니다. 이 모델은 또한 레지스트리에 저장되고 최종적으로 배포가 승인됩니다.

<a id ='train'> </a>
## Architecture for the ML Lifecycle Stage: Train, Check Bias, Tune, Record Lineage, Register Model
[overview](#overview)
___

![train-assess-tune-register](./images/e2e-2-pipeline-v3b.png)

### Install required and/or update libraries

In [ ]:
!python -m pip install -Uq pip
!python -m pip install -q awswrangler==2.2.0 imbalanced-learn==0.7.0 sagemaker==2.23.1 boto3==1.16.48

현재 커널에 업데이트를 적용하려면, 다음 코드를 실행하여 커널을 새로 고칩니다.

In [2]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

### Load stored variables

이전에 이 노트북을 실행한 경우, AWS에서 생성한 리소스를 재사용할 수 있습니다. 아래 셀을 실행하여 이전에 생성된 변수를 로드합니다. 기존 변수의 출력물이 표시되어야 합니다. 인쇄된 내용이 보이지 않으면 노트북을 처음 실행한 것일 수 있습니다.

In [3]:
%store -r
%store

Stored variables and their in-db values:
bucket                        -> 'sagemaker-us-east-1-143656149352'
claims_fg_name                -> 'fraud-detect-demo-claims'
claims_table                  -> 'fraud-detect-demo-claims-1616324730'
col_order                     -> ['fraud', 'policy_state_az', 'policy_annual_premiu
customers_fg_name             -> 'fraud-detect-demo-customers'
customers_table               -> 'fraud-detect-demo-customers-1616324731'
database_name                 -> 'sagemaker_featurestore'
model_data                    -> 's3://sagemaker-us-east-1-143656149352/pytorch-smd
prefix                        -> 'fraud-detect-demo'
test_data_uri                 -> 's3://sagemaker-us-east-1-143656149352/fraud-detec
train_data_uri                -> 's3://sagemaker-us-east-1-143656149352/fraud-detec


**<font color='red'>Important</font>: StoreMagic 명령을 사용하여 변수를 검색하려면 이전 노트북을 실행해야 합니다.**

### Import libraries

In [4]:
import json
import time
import boto3
import sagemaker
import numpy as np
import pandas as pd
import awswrangler as wr

from sagemaker.xgboost.estimator import XGBoost
from model_package_src.inference_specification import InferenceSpecification

### Set region, boto3 and SageMaker SDK variables

In [5]:
#You can change this to a region of your choice
import sagemaker
region = sagemaker.Session().boto_region_name
print("Using AWS Region: {}".format(region))

Using AWS Region: us-east-1


In [6]:
boto3.setup_default_session(region_name=region)

boto_session = boto3.Session(region_name=region)

s3_client = boto3.client('s3', region_name=region)

sagemaker_boto_client = boto_session.client('sagemaker')

sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_boto_client)

sagemaker_role = sagemaker.get_execution_role()

account_id = boto3.client('sts').get_caller_identity()["Account"]

In [7]:
# variables used for parameterizing the notebook run
estimator_output_path = f's3://{bucket}/{prefix}/training_jobs'
train_instance_count = 1
train_instance_type = "ml.m4.xlarge"

bias_report_1_output_path = f's3://{bucket}/{prefix}/clarify-output/bias_1'


xgb_model_name = 'xgb-insurance-claims-fraud-model'
train_instance_count = 1
train_instance_type = "ml.m4.xlarge"
predictor_instance_count = 1
predictor_instance_type = "ml.c5.xlarge"
batch_transform_instance_count = 1
batch_transform_instance_type = "ml.c5.xlarge"
claify_instance_count = 1
clairfy_instance_type = 'ml.c5.xlarge'

<a id='aud-train-model'></a>
## Train a model using XGBoost

[overview](#overview)
___

훈련 및 테스트 데이터셋이 S3에 유지되면 사용할 SageMaker Estimator를 정의하여 모델 훈련을 시작할 수 있습니다. 이 가이드에서는[XGBoost Open Source Framework](https://sagemaker.readthedocs.io/en/stable/frameworks/xgboost/xgboost.html)를 사용하여 모델을 훈련합니다. 이 estimator는 SageMaker SDK를 통해 액세스되지만, [XGBoost Python 패키지](https://xgboost.readthedocs.io/en/latest/python/index.html)의 오픈 소스 버전을 미러링합니다. XGBoost Python 패키지에서 제공하는 모든 기능들은 학습 스크립트에서 구현할 수 있습니다.


### Set the hyperparameters

모델을 훈련시키기 위해 훈련 스크립트로 전송되는 파라메터입니다. 여기에서는 모두 "hyperparameter"로 정의되어 있지만, XGBoost의 [Learning Task Parameters](https://xgboost.readthedocs.io/en/latest/parameter.html#learning-task-parameters)
, [Tree Booster Parameters](https://xgboost.readthedocs.io/en/latest/parameter.html#parameters-for-tree-booster) 또는 XGBoost에 대해 설정하려는 기타 파라메터를 포함할 수 있습니다.

In [8]:
hyperparameters = {
        "max_depth": "3",
        "eta": "0.2",
        "objective": "binary:logistic",
        "num_round": "100",
}
%store hyperparameters

Stored 'hyperparameters' (dict)


### Create and fit the estimator
SageMaker XGBoost Framework에서 제공하는 광범위한 함수들을 탐색하려는 경우, 상속 클래스를 참조하여 모든 설정 파라메터에 대해 읽을 수 있습니다. XGBoost 클래스는 Framework 클래스에서 상속되고 Framework는 EstimatorBase 클래스에서 상속됩니다.

* [XGBoost Estimator documentation](https://sagemaker.readthedocs.io/en/stable/frameworks/xgboost/xgboost.html#sagemaker.xgboost.estimator.XGBoost)
* [Framework documentation](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.Framework)
* [EstimatorBase documentation](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase)

In [9]:
xgb_estimator = XGBoost(
    entry_point = "xgboost_starter_script.py",
    output_path = estimator_output_path,
    code_location = estimator_output_path,
    hyperparameters = hyperparameters,
    role = sagemaker_role,
    instance_count = train_instance_count,
    instance_type = train_instance_type,
    framework_version = "1.0-1")

In [10]:
if 'training_job_1_name' not in locals():
    
    xgb_estimator.fit(inputs = {'train': train_data_uri})
    training_job_1_name = xgb_estimator.latest_training_job.job_name
    %store training_job_1_name
    
else:
    print(f'Using previous training job: {training_job_1_name}')

2021-03-21 11:18:46 Starting - Starting the training job...
2021-03-21 11:19:09 Starting - Launching requested ML instancesProfilerReport-1616325525: InProgress
......
2021-03-21 11:20:09 Starting - Preparing the instances for training......
2021-03-21 11:21:09 Downloading - Downloading input data...
2021-03-21 11:21:29 Training - Downloading the training image...
2021-03-21 11:22:16 Training - Training image download completed. Training in progress..INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Invoking user training script.
INFO:sagemaker-containers:Module xgboost_starter_script does not provide a setup.py. 
Generating setup.py
INFO:sagemaker-containers:Generating setup.cfg
INFO:sagemaker-containers:Generating MANIFEST.in
INFO:sagemaker-containers:Installing module with the following command:
/miniconda3/bin/python3 -m pip install . 
P

<a id='model-lineage'></a>
## Model lineage with artifacts and associations

[Overview](#aud-overview)
___

Amazon SageMaker ML Lineage Tracking은 데이터 준비에서 모델 배포에 이르는 머신 러닝 (ML) 워크플로의 단계(step)에 대한 정보를 생성하고 저장합니다. 추적(tracking) 정보를 사용하여 워크플로 단계를 재현하고, 모델 및 데이터 집합 계보를 추적하고, 모델 거버넌스 및 감사 표준을 설정할 수 있습니다. SageMaker Lineage Tracking을 통해 데이터 과학자 및 모델러는 아래 작업들을 수행할 수 있습니다.

* 모델 발견 실험의 실행 기록을 유지합니다.
* 감사 및 규정 준수 확인을 위해 모델 계보 아티팩트를 추적하여 모델 거버넌스를 설정합니다.
* 워크플로를 복제하고 다시 실행하여 모델을 개발하는 동안 가상 시나리오를 실험합니다.
* 동료가 재현하고 향상시킬 수 있는 워크플로를 공유합니다. (예: 비즈니스 문제 해결을 위해 공동 작업하는 동안)
* 추가 디버깅 또는 로깅 루틴을 사용하여 워크플로를 복제하고 다시 실행하거나, 프로덕션 모델의 문제를 해결하기 위한 새로운 입력 변형을 사용합니다.

<a id='register-artifacts'></a>
### Register artifacts

`xgb_estimator` 객체는 모델 훈련 방법에 대해 훈련하는 데 필요한 많은 데이터를 보유하고 있지만, 실제로는 SageMaker가 지속되지 않고 나중에 다시 인스턴스화 할 수 없는 임시 객체입니다. 연결이 사라지면 일부 연결을 잃어도 일단 생성된 훈련 작업에 액세스하여 필요한 모든 데이터를 다시 가져올 수 있습니다.

In [11]:
training_job_1_info = sagemaker_boto_client.describe_training_job(TrainingJobName=training_job_1_name)

#### Code artifact

In [12]:
# return any existing artifact which match the our training job's code arn
# ====>

# extract the training code uri and check if it's an exisiting artifact
code_s3_uri = training_job_1_info['HyperParameters']['sagemaker_submit_directory']

matching_artifacts = list(sagemaker.lineage.artifact.Artifact.list(
    source_uri=code_s3_uri, 
    sagemaker_session=sagemaker_session))

# use existing arifact if it's already been created, otherwise create a new artifact
if matching_artifacts:
    code_artifact = matching_artifacts[0]
    print(f'Using existing artifact: {code_artifact.artifact_arn}')
else:
    code_artifact = sagemaker.lineage.artifact.Artifact.create(
        artifact_name='TrainingScript',
        source_uri=code_s3_uri,
        artifact_type='Code',
        sagemaker_session=sagemaker_session)
    print(f'Create artifact {code_artifact.artifact_arn}: SUCCESSFUL')

Create artifact arn:aws:sagemaker:us-east-1:143656149352:artifact/5dcb74b844a00c2a8343d3e5da55c492: SUCCESSFUL


#### Training data artifact

In [13]:
training_data_s3_uri = training_job_1_info['InputDataConfig'][0]['DataSource']['S3DataSource']['S3Uri']

matching_artifacts = list(sagemaker.lineage.artifact.Artifact.list(
    source_uri=training_data_s3_uri,
    sagemaker_session=sagemaker_session))

if matching_artifacts:
    training_data_artifact = matching_artifacts[0]
    print(f'Using existing artifact: {training_data_artifact.artifact_arn}')
else:
    training_data_artifact = sagemaker.lineage.artifact.Artifact.create(
        artifact_name='TrainingData',
        source_uri=training_data_s3_uri,
        artifact_type='Dataset',
        sagemaker_session=sagemaker_session)
    print(f'Create artifact {training_data_artifact.artifact_arn}: SUCCESSFUL')

Using existing artifact: arn:aws:sagemaker:us-east-1:143656149352:artifact/a862b5126a105e131245bb544555d2dd


#### Model artifact

In [14]:
trained_model_s3_uri = training_job_1_info['ModelArtifacts']['S3ModelArtifacts']

matching_artifacts = list(sagemaker.lineage.artifact.Artifact.list(
    source_uri=trained_model_s3_uri,
    sagemaker_session=sagemaker_session))

if matching_artifacts:
    model_artifact = matching_artifacts[0]
    print(f'Using existing artifact: {model_artifact.artifact_arn}')
else:
    model_artifact = sagemaker.lineage.artifact.Artifact.create(
        artifact_name='TrainedModel',
        source_uri=trained_model_s3_uri,
        artifact_type='Model',
        sagemaker_session=sagemaker_session)
    print(f'Create artifact {model_artifact.artifact_arn}: SUCCESSFUL')

Using existing artifact: arn:aws:sagemaker:us-east-1:143656149352:artifact/b2d1e550b3665bdaaebd25985da56549


<a id='Set-artifact-associations'></a>
### Set artifact associations

In [15]:
trial_component = sagemaker_boto_client.describe_trial_component(TrialComponentName=training_job_1_name+'-aws-training-job')
trial_component_arn = trial_component['TrialComponentArn']

#### Input artifacts

In [16]:
input_artifacts = [code_artifact, training_data_artifact]

for a in input_artifacts:
    try:
        sagemaker.lineage.association.Association.create(
            source_arn=a.artifact_arn,
            destination_arn=trial_component_arn,
            association_type='ContributedTo',
            sagemaker_session=sagemaker_session)
        print(f"Association with {a.artifact_type}: SUCCEESFUL")
    except:
        print(f"Association already exists with {a.artifact_type}")

Association with Code: SUCCEESFUL
Association already exists with DataSet


#### Output artifacts

In [17]:
output_artifacts = [model_artifact]

for a in output_artifacts:
    try:
        sagemaker.lineage.association.Association.create(
            source_arn=a.artifact_arn,
            destination_arn=trial_component_arn,
            association_type='Produced',
            sagemaker_session=sagemaker_session)
        print(f"Association with {a.artifact_type}: SUCCESSFUL")
    except:
        print(f"Association already exists with {a.artifact_type}")

Association with Model: SUCCESSFUL


<a id='check-bias'></a>
## Evaluate model for bias with Clarify

[overview](#aud-overview)
___

Amazon SageMaker Clarify는 잠재적인 편향(bias)을 감지하고 모델의 예측을 설명하여 머신 러닝 (ML) 모델을 개선하는 데 도움이 됩니다. 사전 훈련 데이터와 모델 훈련 중 또는 모델이 프로덕션 중일 때 나타날 수 있는 사후 훈련에서 다양한 유형의 편향을 식별하는 데 도움이 됩니다. SageMaker Clarify는 이러한 모델이 피쳐 속성 접근 방식을 사용하여 예측하는 방법을 설명하는 데 도움이 됩니다. 또한 프로덕션에서 편향 또는 feature attribution drift에 대한 모델의 추론을 모니터링합니다. SageMaker Clarify에서 제공하는 공정성과 설명 가능성 기능은 AWS 고객이 편향이 덜하고 이해하기 쉬운 머신 러닝 모델을 구축하는 데 도움 이되는 컴포넌트들을 제공합니다. 또한 위험 및 규정 준수 팀과 외부 규제 기관에 알리는 데 사용할 수 있는 모델 거버넌스 보고서를 생성하는 데 도움이 되는 도구를 제공합니다.

SageMaker Clarify에 대한 자세한 내용은 [SageMaker Developer Guide](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-fairness-and-explainability.html)를 참조하십시오.

### Create model from estimator

In [18]:
model_1_name = f'{prefix}-xgboost-pre-smote'
%store model_1_name
model_matches = sagemaker_boto_client.list_models(NameContains=model_1_name)['Models']

if not model_matches:
    
    model_1 = sagemaker_session.create_model_from_job(
        name=model_1_name,
        training_job_name=training_job_1_info['TrainingJobName'],
        role=sagemaker_role,
        image_uri=training_job_1_info['AlgorithmSpecification']['TrainingImage'])
else:
    
    print(f"Model {model_1_name} already exists.")

Stored 'model_1_name' (str)


<a id='bias-v1'></a>
### Check for data set bias and model bias

SageMaker를 사용하면 사전 훈련 및 사후 훈련 편향을 확인할 수 있습니다. 사전 훈련 metric은 해당 데이터의 기존 metric을 보여주는 반면, 사후 훈련 metric은 모델의 예측에서 편향을 보여줍니다. SageMaker SDK를 사용하여 편향을 확인하려는 그룹과 표시할 metric을 지정할 수 있습니다.

전체 Clarify 작업을 실행하려면, 아래 셀에서 코드의 주석 처리를 제거해야 합니다. 작업을 실행하는 데 약 15분 정도 소요됩니다. 시간을 절약하려면 편향 작업이 실행되지 않은 경우 미리 생성된 결과를 로드한 후, 다음 셀에서 결과를 볼 수 있습니다.

In [19]:
train_cols = wr.s3.read_csv(training_data_s3_uri).columns.to_list()

clarify_processor = sagemaker.clarify.SageMakerClarifyProcessor(
    role=sagemaker_role,
    instance_count=1,
    instance_type='ml.c4.xlarge',
    sagemaker_session=sagemaker_session)

bias_data_config = sagemaker.clarify.DataConfig(
    s3_data_input_path=train_data_uri,
    s3_output_path=bias_report_1_output_path,
    label='fraud',
    headers=train_cols,
    dataset_type='text/csv')

model_config = sagemaker.clarify.ModelConfig(
    model_name=model_1_name,
    instance_type=train_instance_type,
    instance_count=1,
    accept_type='text/csv')

predictions_config = sagemaker.clarify.ModelPredictedLabelConfig(probability_threshold=0.5)

bias_config = sagemaker.clarify.BiasConfig(
    label_values_or_threshold=[0],
    facet_name='customer_gender_female',
    facet_values_or_threshold=[1])

# un-comment the code below to run the whole job

# if 'clarify_bias_job_1_name' not in locals():

#     clarify_processor.run_bias(
#         data_config=bias_data_config,
#         bias_config=bias_config,
#         model_config=model_config,
#         model_predicted_label_config=predictions_config,
#         pre_training_methods='all',
#         post_training_methods='all')

#     clarify_bias_job_1_name = clarify_processor.latest_job.name
#     %store clarify_bias_job_1_name

# else:
#     print(f'Clarify job {clarify_bias_job_name} has already run successfully.')

결과는 `/opt/ml/processing/output/report.pdf`에 저장됩니다. 90% 이상의 분류 정확도를 달성하기 위한 훈련은 불균형 분류 문제에서 쉽게 발생하며, 이는 잘못된 가정과 결론으로 이어집니다. 데이터 과학자와 이해관계자는 모델이 실제로는 그렇지 않은데도 성능이 극도로 높다고 믿도록 오해합니다.

### View results of Clarify job (shortcut)

데이터셋 또는 모델에서 Clarify를 실행하는 데 15분 정도 걸릴 수 있습니다. 작업을 실행할 시간이 없는 경우, 이 데모에 포함된 미리 생성된 결과를 볼 수 있습니다. 그렇지 않으면 위의 셀에서 코드 주석 처리를 제거하여 작업을 실행할 수 있습니다. 

In [20]:
if 'clarify_bias_job_name' in locals():
    s3_client.download_file(Bucket=bucket, Key=f'{prefix}/clarify-output/bias-1/analysis.json', Filename='clarify_output/bias_1/analysis.json')
    print(f'Downloaded analysis from previous Clarify job: {clarify_bias_job_name}')
else:
    print(f'Loading pre-generated analysis file...')

with open('clarify_output/bias_1/analysis.json', 'r') as f:
        bias_analysis = json.load(f)

results = bias_analysis['pre_training_bias_metrics']['facets']['customer_gender_female'][0]['metrics'][1]
print(json.dumps(results, indent=4))

Loading pre-generated analysis file...
{
    "name": "CI",
    "description": "Class Imbalance (CI)",
    "value": 0.398
}


이 예제 데이터셋에서 데이터는 여성 고객의 데이터 샘플 중 38.9%에 불과한 여성에 대해 편향되어 있습니다. 이 클래스 불균형 편향을 완화하는 방법을 보여주는 다음 노트북에서 이 문제를 다룰 것입니다. 클래스 불균형은 편향 통계의 한 예일 뿐이지만, 다른 많은 편향 요인도 고려할 수 있습니다. 자세한 내용은 [Fairness Measures for Machine Learning in Finance](https://pages.awscloud.com/rs/112-TZM-766/images/Fairness.Measures.for.Machine.Learning.in.Finance.pdf)을 참조하십시오.

더 자세한 예제는 [이 GitHub 예제](https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker_processing/fairness_and_explainability/fairness_and_explainability.ipynb)를
참조하십시오.

더 자세한 결과는 S3 버킷에 생성된 보고서에서 확인이 가능합니다: `s3://{bucket}/e2e-fraud-detect/clarify/bias-2/report.pdf`<br>
보고서를 확인하려면 아래 코드 셀의 주석을 해제해 주세요. (단, 위에서 Clarify를 실행하지 않았다면, 보고서가 생성되지 않습니다.) 

In [32]:
#uncomment to copy report and view
#!aws s3 cp s3://{bucket}/e2e-fraud-detect/clarify/bias-2/report.pdf ./clarify_output

<a id='model-registry'></a>
## Deposit Model and Lineage in SageMaker Model Registry

[overview](#aud-overview)
____
유용한 모델이 훈련되고 해당 아티팩트가 적절하게 연결되면, 다음 단계는 향후 참조 및 가능한 배포를 위해 모델을 레지스트리에 저장하는 것입니다.

### Create Model Package Group

모델 패키지 그룹은 모델의 여러 버전 또는 iteration을 보유합니다. 레지스트리의 모든 모델에 대해 생성할 필요는 없지만 모두 동일한 목적을 갖고 자동 버전 관리를 제공하는 다양한 모델을 구성하는 데 도움이 됩니다.

In [22]:
if 'mpg_name' not in locals():
    mpg_name = prefix
    %store mpg_name
    print(f'Model Package Group name: {mpg_name}')

Stored 'mpg_name' (str)
Model Package Group name: fraud-detect-demo


In [23]:
mpg_input_dict = {
    'ModelPackageGroupName': mpg_name,
    'ModelPackageGroupDescription': 'Insurance claim fraud detection'
}

In [24]:
matching_mpg = sagemaker_boto_client.list_model_package_groups(NameContains=mpg_name)['ModelPackageGroupSummaryList']

if matching_mpg:
    print(f'Using existing Model Package Group: {mpg_name}')
else:
    mpg_response = sagemaker_boto_client.create_model_package_group(**mpg_input_dict)
    print(f'Create Model Package Group {mpg_name}: SUCCESSFUL')
    %store mpg_name

Create Model Package Group fraud-detect-demo: SUCCESSFUL
Stored 'mpg_name' (str)


### Create Model Package for trained model

#### Create and upload a metrics report

In [25]:
model_metrics_report = {'classification_metrics': {}}
for metric in training_job_1_info['FinalMetricDataList']:
    stat = {metric['MetricName']: {'value': metric['Value']}}
    model_metrics_report['classification_metrics'].update(stat)
    
with open('training_metrics.json', 'w') as f:
    json.dump(model_metrics_report, f)
    
metrics_s3_key = f"{prefix}/training_jobs/{training_job_1_info['TrainingJobName']}/training_metrics.json"
s3_client.upload_file(Filename='training_metrics.json', Bucket=bucket, Key=metrics_s3_key)

#### Define the inference spec

In [26]:
mp_inference_spec = InferenceSpecification().get_inference_specification_dict(
    ecr_image=training_job_1_info['AlgorithmSpecification']['TrainingImage'],
    supports_gpu=False,
    supported_content_types=['text/csv'],
    supported_mime_types=['text/csv'])

mp_inference_spec['InferenceSpecification']['Containers'][0]['ModelDataUrl'] = training_job_1_info['ModelArtifacts']['S3ModelArtifacts']

#### Define model metrics

모델 품질 및 편향 이외의 메트릭을 정의할 수 있습니다. 모델 패키지를 생성하려면 [Boto3 설명서](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_model_package)를 참조하십시오.

In [27]:
model_metrics = {
    'ModelQuality': {
        'Statistics': {
            'ContentType': 'application/json',
            'S3Uri': f's3://{bucket}/{prefix}/{metrics_s3_key}'
        }
    },
    'Bias': {
        'Report': {
            'ContentType': 'application/json',
            'S3Uri': f'{bias_report_1_output_path}/analysis.json'
        }
    }
}

In [28]:
mp_input_dict = {
    'ModelPackageGroupName': mpg_name,
    'ModelPackageDescription': 'XGBoost classifier to detect insurance fraud.',
    'ModelApprovalStatus': 'PendingManualApproval',
    'ModelMetrics': model_metrics
}

mp_input_dict.update(mp_inference_spec)
mp1_response = sagemaker_boto_client.create_model_package(**mp_input_dict)

### Wait until model package is completed

In [29]:
mp_info = sagemaker_boto_client.describe_model_package(ModelPackageName=mp1_response['ModelPackageArn'])
mp_status = mp_info['ModelPackageStatus']

while mp_status not in ['Completed', 'Failed']:
    time.sleep(5)
    mp_info = sagemaker_boto_client.describe_model_package(ModelPackageName=mp1_response['ModelPackageArn'])
    mp_status = mp_info['ModelPackageStatus']
    print(f'model package status: {mp_status}')
print(f'model package status: {mp_status}')

model package status: Completed


### View model package in registry

In [30]:
sagemaker_boto_client.list_model_packages(ModelPackageGroupName=mpg_name)['ModelPackageSummaryList']

[{'ModelPackageGroupName': 'fraud-detect-demo',
  'ModelPackageVersion': 1,
  'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:143656149352:model-package/fraud-detect-demo/1',
  'ModelPackageDescription': 'XGBoost classifier to detect insurance fraud.',
  'CreationTime': datetime.datetime(2021, 3, 21, 11, 25, 58, 732000, tzinfo=tzlocal()),
  'ModelPackageStatus': 'Completed',
  'ModelApprovalStatus': 'PendingManualApproval'}]

___

### Next Notebook: [Mitigate Bias, Train New Model, Store in Registry](./3-mitigate-bias-train-model2-registry-e2e.ipynb)

불균형을 처리하기 위해, 다음 노트북에서 [SMOTE (Synthetic Minority Over-sampling Technique)](https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.over_sampling.SMOTE.html)를 사용하여 소수 클래스를 오버샘플링 (즉, 업 샘플링)합니다.